<center><div style="width: 900px;  padding-top:5px; padding-bottom:10px;border: 3px solid #1625CB; text-align: left;background: #1625CB;">
 <center>DataFrame.groupeby </center>
</div></center>


L'opération groupby est essentielle à l'agrégation et à la synthèse des données dans les pipelines ETL. 
Elle permet de :

-  Agréger les données à différents niveaux : synthétiser par produit, client, région, période, etc.
-  Appliquer plusieurs fonctions d'agrégation : calculer la somme, la moyenne, le nombre, le minimum, le maximum et des indicateurs personnalisés
-  Créer des regroupements hiérarchiques : regrouper selon plusieurs dimensions pour une analyse multiniveau
-  Transformer les données : réintégrer les valeurs agrégées au DataFrame d'origine
-  Filtrer les groupes : conserver uniquement les groupes répondant à certains critères

Dans les workflows ETL, groupby est souvent utilisé pour préparer les données pour les rapports, créer des tableaux récapitulatifs pour les entrepôts de données ou générer des fonctionnalités pour les modèles de machine learning. 

La flexibilité de définition de fonctions d'agrégation personnalisées le rend particulièrement performant pour les logiques métier complexes.

In [ ]:
import pandas as pd
import numpy as np


# Données de ventes fictives
sales = pd.DataFrame({
    'date': pd.date_range(start='2023-01-01', periods=20),
    'product': ['A', 'B', 'A', 'C', 'B', 'A', 'C', 'B', 'A', 'C',
                'A', 'B', 'C', 'A', 'B', 'C', 'A', 'B', 'C', 'A'],
    'region': ['Est', 'Ouest', 'Est', 'Nord', 'Sud', 'Ouest', 'Est', 'Nord', 'Sud', 'Ouest',
               'Est', 'Ouest', 'Nord', 'Sud', 'Est', 'Ouest', 'Nord', 'Sud', 'Est', 'Ouest'],
    'units': np.random.randint(1, 50, 20),  # Quantité vendue aléatoire
    'price': np.random.uniform(10, 100, 20).round(2)  # Prix unitaire aléatoire
})
sales

,date,product,region,units,price
0,2023-01-01,A,Est,14,24.25
1,2023-01-02,B,Ouest,1,10.29
2,2023-01-03,A,Est,40,27.41
3,2023-01-04,C,Nord,35,78.63
4,2023-01-05,B,Sud,44,29.80
5,2023-01-06,A,Ouest,37,10.34
6,2023-01-07,C,Est,12,72.57
7,2023-01-08,B,Nord,32,65.65
8,2023-01-09,A,Sud,24,52.77
9,2023-01-10,C,Ouest,37,20.52


In [14]:
# Calcul du chiffre d'affaires 
sales['revenue'] = sales['units'] * sales['price']

In [15]:
# Regroupement de base avec une seule colonne et une seule agrégation
product_sales = sales.groupby('product')['revenue'].sum()
print("Chiffre d'affaires total par produit :")
print(product_sales)

Chiffre d'affaires total par produit :
product
A    7384.85
B    6492.84
C    8745.33
Name: revenue, dtype: float64


In [16]:
# Regroupement avec plusieurs agrégations
product_stats = sales.groupby('product').agg({
    'revenue': ['sum', 'mean', 'count'],
    'units': ['sum', 'mean', 'min', 'max']
})
print("\nStatistiques par produit :")
print(product_stats)


Statistiques par produit :
         revenue                   units                   
             sum        mean count   sum       mean min max
product                                                    
A        7384.85   923.10625     8   191  23.875000   2  40
B        6492.84  1082.14000     6   165  27.500000   1  47
C        8745.33  1457.55500     6   164  27.333333  12  45


In [17]:
# Regroupement avec plusieurs colonnes
region_product_sales = sales.groupby(['region', 'product'])['revenue'].sum()
print("\nChiffre d'affaires par région et produit :")
print(region_product_sales)


Chiffre d'affaires par région et produit :
region  product
Est     A          3151.01
        B            51.05
        C          1187.94
Nord    A           198.52
        B          2100.80
        C          6104.55
Ouest   A          1682.84
        B           627.87
        C          1452.84
Sud     A          2352.48
        B          3713.12
Name: revenue, dtype: float64


In [18]:

# Regroupement avec une fonction d’agrégation personnalisée
def revenue_per_unit(x):
    return x['revenue'].sum() / x['units'].sum()

custom_agg = sales.groupby('product').apply(revenue_per_unit)
print("\nAgrégation personnalisée - Chiffre d'affaires par unité pour chaque produit :")
print(custom_agg)


Agrégation personnalisée - Chiffre d'affaires par unité pour chaque produit :
product
A    38.664136
B    39.350545
C    53.325183
dtype: float64


C:\Users\papes\AppData\Local\Temp\ipykernel_24284\1216626984.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  custom_agg = sales.groupby('product').apply(revenue_per_unit)


In [19]:
# Regroupement avec transformation (conserve la forme du DataFrame d'origine)
sales['product_avg_price'] = sales.groupby('product')['price'].transform('mean')
print("\nDataFrame avec une colonne transformée (prix moyen par produit) :")
print(sales[['product', 'price', 'product_avg_price']].head(10))



DataFrame avec une colonne transformée (prix moyen par produit) :
  product  price  product_avg_price
0       A  24.25             51.555
1       B  10.29             32.635
2       A  27.41             51.555
3       C  78.63             50.340
4       B  29.80             32.635
5       A  10.34             51.555
6       C  72.57             50.340
7       B  65.65             32.635
8       A  52.77             51.555
9       C  20.52             50.340


In [20]:
# Regroupement avec filtrage
high_volume_products = sales.groupby('product').filter(lambda x: x['units'].sum() > 100)
print("\nProduits à fort volume (>100 unités vendues au total) :")
print(high_volume_products['product'].unique())


Produits à fort volume (>100 unités vendues au total) :
['A' 'B' 'C']


In [21]:
# Regroupement basé sur le temps (par mois)
sales['month'] = sales['date'].dt.month
monthly_sales = sales.groupby('month')['revenue'].sum()
print("\nChiffre d'affaires mensuel :")
print(monthly_sales)


Chiffre d'affaires mensuel :
month
1    22623.02
Name: revenue, dtype: float64


In [22]:
# Regroupement avec tableau croisé dynamique (approche alternative)
pivot_table = pd.pivot_table(
    sales, 
    values='revenue', 
    index='region', 
    columns='product', 
    aggfunc='sum',
    fill_value=0
)
print("\nTableau croisé du chiffre d'affaires par région et produit :")
print(pivot_table)


Tableau croisé du chiffre d'affaires par région et produit :
product        A        B        C
region                            
Est      3151.01    51.05  1187.94
Nord      198.52  2100.80  6104.55
Ouest    1682.84   627.87  1452.84
Sud      2352.48  3713.12     0.00


In [ ]:
L'opération groupby est essentielle à l'agrégation et à la synthèse des données dans les pipelines ETL. Elle permet de :
Agréger les données à différents niveaux : synthétiser par produit, client, région, période, etc.
Appliquer plusieurs fonctions d'agrégation : calculer la somme, la moyenne, le nombre, le minimum, le maximum et des indicateurs personnalisés
Créer des regroupements hiérarchiques : regrouper selon plusieurs dimensions pour une analyse multiniveau
Transformer les données : réintégrer les valeurs agrégées au DataFrame d'origine
Filtrer les groupes : conserver uniquement les groupes répondant à certains critères
Dans les workflows ETL, groupby est souvent utilisé pour préparer les données pour les rapports, créer des tableaux récapitulatifs pour les entrepôts de données ou générer des fonctionnalités pour les modèles de machine learning. La flexibilité de définition de fonctions d'agrégation personnalisées le rend particulièrement performant pour les logiques métier complexes.

SyntaxError: invalid non-printable character U+00A0 (390026360.py, line 1)